In [41]:
import math
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from tqdm import tqdm

import os
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import time

USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial
device = torch.device('cuda') if (USE_GPU and torch.cuda.is_available()) else torch.device('mps')
print('using device:', device)

using device: mps


## American Option Pricing with Multiple Neural Networks (method 1) article [1]
Here I'll try a simple implementation of the method I of the first article :

Here we have constant interest rate so the discount factor is $\exp(-rT)$, and the stock dynamics are modelled with Geometric Brownian Motion (GBM).

$\large dS_t = rS_tdt+\sigma S_tdW_t$

Let's simulate this GBM process by simulating variables of the natural logarithm process of the stock price $x_t = ln(S_t)$, which is normally distributed. For the dynamics of the natural logarithm process of stock prices under GBM model you need to use Ito's calculus.
$\large dx_t = \nu dt+\sigma dW_t ,  \nu = r - \frac{1}{2} \sigma ^ 2$

We can then discretize the Stochastic Differential Equation (SDE) by changing the infinitesimals $dx, dt, dz$ into small steps $\Delta x, \Delta t, \Delta W$.

$\large \Delta x = \nu  \Delta t+\sigma \Delta W$

This is the exact solution to the SDE and involves no approximation.

$\large x_{t+\Delta t} = x_{t} + \nu (\Delta t)+\sigma (W_{t+\Delta t}- W_t)$

In terms of the stock price S, we have:

$\large S_{t+\Delta t} = S_{t} \exp( \nu \Delta t + \sigma (W_{t+\Delta t}- W_t) )$

Where $(W_{t+\Delta t}- W_t) \sim N(0,\Delta t) \sim \sqrt{\Delta t} N(0,1) \sim \sqrt{\Delta t} \epsilon_i$


\\


# The algorithm :

***Algorithm 1 :*** American Option Pricing with Multiple Neural Networks

**Result :** Functions $\Phi_{t_i}, \Psi_{t_i}$ for $i \in \{0,1, \ldots, n-1\}$

Simulate $N$ stock paths

Initialize $Y_{t_n} = X_{t_n} = f\left(S_{t_n}\right)$

for $i = n-1 : 1$ do  
&nbsp;&nbsp;&nbsp;&nbsp; Regress $\beta_{\Delta t}^{-1} Y_{t_{i+1}}$ on $S_{t_i}$:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\min_{\Phi_{t_i}, \Psi_{t_i}}\left(\beta_{\Delta t}^{-1} Y_{t_{i+1}} - \Phi_{t_i}\left(S_{t_i}\right) - \Psi_{t_i}\left(S_{t_i}\right) \Delta W_{t_i}\right)^2$  
&nbsp;&nbsp;&nbsp;&nbsp; $Y_{t_i} = \beta_{\Delta t}^{-1} Y_{t_{i+1}} - \Psi_{t_i}\left(S_{t_i}\right) \Delta W_{t_i}$  
&nbsp;&nbsp;&nbsp;&nbsp; $X_{t_i} = \beta_{\Delta t}^{-1} X_{t_{i+1}} - \Psi_{t_i}\left(S_{t_i}\right) \Delta W_{t_i}$  
&nbsp;&nbsp;&nbsp;&nbsp; if $f\left(S_{t_i}\right) > \Phi_{t_i}\left(S_{t_i}\right)$ then  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $Y_{t_i} = f\left(S_{t_i}\right)$  
&nbsp;&nbsp;&nbsp;&nbsp; end  
&nbsp;&nbsp;&nbsp;&nbsp; if $f\left(S_{t_i}\right) > X_{t_i}$ then  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $X_{t_i} = f\left(S_{t_i}\right)$  
&nbsp;&nbsp;&nbsp;&nbsp; end  
end

Regress $\beta_{\Delta t}^{-1} Y_{t_1}$ on $S_{t_0}$:  
&nbsp;&nbsp;&nbsp;&nbsp; $\min \left(\beta_{\Delta t}^{-1} Y_{t_1} - \Phi_{t_0}\left(S_{t_0}\right) - \Psi_{t_0}\left(S_{t_0}\right) \Delta W_{t_0}\right)^2$  
&nbsp;&nbsp;&nbsp;&nbsp; $Y_{t_0} = \beta_{\Delta t}^{-1} Y_{t_1} - \Psi_{t_0}\left(S_{t_0}\right) \Delta W_{t_0}$  
&nbsp;&nbsp;&nbsp;&nbsp; $X_{t_0} = \beta_{\Delta t}^{-1} X_{t_1} - \Psi_{t_0}\left(S_{t_0}\right) \Delta W_{t_0}$  


In [42]:
class stochastic_models() :
    def __init__(self, r, vol , S , K, T , dt , n , M) :
        self.r  = r
        self.vol =  vol
        self.S = S
        self.K = K
        self.dt = dt
        self.n = n
        self.T = T
        self.M = M
        self.beta_dt = math.exp(-self.r*self.dt)

        self.S_simulation = None
        self.dW_simulation = None

    def simulate_path(self , device ) :
        nudt = (self.r - 0.5 * self.vol**2) * self.dt
        lnS = np.log(self.S)

        # Méthode de Monte Carlo
        Z = np.random.normal(size=(self.M,self.n))
        dW=np.sqrt(self.dt) * Z  #it's the simulation of the dWt_i we'll need in each iteration
        delta_lnSt = nudt + self.vol*dW
        LnS_s = np.zeros([self.M, self.n + 1])

        for i in range(self.M):
            LnS_s[i, 0] = lnS
            for j in range(1, n + 1):
                LnS_s[i, j] = LnS_s[i, j - 1] + delta_lnSt[i,j - 1]

        S = np.exp(LnS_s)
        S_tensor = torch.tensor(S, device = device ,dtype=dtype)
        dW_tensor = torch.tensor(dW, device = device ,dtype=dtype)
        self.S_simulation, self.dW_simulation =  S_tensor,dW_tensor
        

#Parametres
T = 1
n = 50
dt = T/n  #les t_i seront donc les i*dt.


S = 36          # Prix de l'action
K = 40           # Prix d'exercice
vol = 0.2       # Volatilité (%)
r = 0.06            # Taux sans risque (%)
M = 20000        # Nombre de simulations

stochastic_model = stochastic_models(r = r, vol = vol, S = S, K = K, n = n , dt = dt , T = T , M = M)
stochastic_model.simulate_path(device = device)

In [47]:
class IterationModel(nn.Module):
    def __init__(self):
        super(IterationModel, self).__init__()
        self.branch1 = nn.Sequential(
            nn.Linear(1, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 1)
        )
        self.branch2 = nn.Sequential(
            nn.Linear(1, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 1) ,          
  
        )
        self.branch3 = nn.Sequential(
            nn.Linear(1, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 1) ,          
        )


    def forward(self, x):
        x_branch1 = self.branch1(x)
        x_branch2 = self.branch2(x)
        x_branch3 = self.branch3(x)
        
        concatenated_output = torch.cat((x_branch1, x_branch2 , x_branch3), dim=1)
        return concatenated_output


class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0):
        """
        Args:
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                pass
                # print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0



class RecurrentNetworkModel(nn.Module) :
    def __init__(self  , lr = 0.01 , device = device , stochastic_model = stochastic_model):
        super(RecurrentNetworkModel, self).__init__()
        self.model = IterationModel().to(device) 
        self.K = K
        self.stochastic_model = stochastic_model
        self.beta_dt = self.stochastic_model.beta_dt
        self.K = self.stochastic_model.K
        self.n_iterations = self.stochastic_model.n
        self.optimizer =optim.Adam(self.model.parameters(), lr=lr) # type: ignore
        self.criterion = nn.MSELoss()
        self.S , self.dW =  self.stochastic_model.S_simulation, self.stochastic_model.dW_simulation



    def custom_loss(self , Y_true, dW_true ,dW_true_2 , y_pred):
        return  self.criterion(y_pred[:, 0] + dW_true * y_pred[:, 1] + (dW_true_2-dt)*y_pred[:, 2], Y_true)
    
    def eval_all(self, iter, S_iter, batch_size ,X,  Y , dW_iter ) :
        X_new = []
        Y_new = []

        eval_dataset = TensorDataset(S_iter, dW_iter, X, Y)
        eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)
        self.model.eval()
        with torch.no_grad() :
            if iter > 0 :
                for S_batch, dW_batch, X_batch, Y_batch in eval_loader:
                    Fi = self.model(S_batch.unsqueeze(1))
                    Z_batch = F.relu(self.K- S_batch)  
                    dW_batch_square = dW_batch*dW_batch
                    Y_batch = torch.where(Z_batch> Fi[:,0], Z_batch, self.beta_dt*Y_batch-Fi[:,1]*dW_batch - Fi[:,2]*dW_batch_square)
                    X_batch = torch.where(Z_batch> X_batch, Z_batch, self.beta_dt*X_batch-Fi[:,1]*dW_batch - Fi[:,2]*dW_batch_square) 
                    X_new.append(X_batch)
                    Y_new.append(Y_batch)
            else :
                for S_batch, dW_batch, X_batch, Y_batch in eval_loader:
                    Fi = self.model(S_batch.unsqueeze(1))
                    Z_batch = F.relu(self.K- S_batch)  
                    dW_batch_square = dW_batch*dW_batch
                    Y_batch =  self.beta_dt*Y_batch-Fi[:,1]*dW_batch  - Fi[:,2]*dW_batch_square
                    X_batch =  self.beta_dt*X_batch-Fi[:,1]*dW_batch  - Fi[:,2]*dW_batch_square
                    X_new.append(X_batch)
                    Y_new.append(Y_batch)
        
        X = torch.cat(X_new, dim=0)
        Y = torch.cat(Y_new, dim=0)
        return X, Y

    def train(self ,batch_size  = 2000, batch_size_eval = 256 , num_epochs = 5 , valid_split = 0.2 , patience = 10) : 
        S , dW =  self. S, self.dW
        X = F.relu(self.K- S[:,self.n_iterations])
        Y = F.relu(self.K- S[:,self.n_iterations])
        # early_stopping = EarlyStopping(patience=patience ,verbose=True)
        train_size = int((1 - valid_split) * S.size(0))
        val_size = S.size(0) - train_size
        train_idxs, val_idxs = torch.utils.data.random_split(range(S.size(0)), [train_size, val_size])
        write = True
        for iter in tqdm(range(self.n_iterations-1,-1,-1), desc="Processing Iterations") :
            early_stopping = EarlyStopping(patience=patience ,verbose=True)

            S_iter, dW_iter = S[:,iter], dW[:,iter]
            S_iter_train, dW_iter_train = S_iter[train_idxs.indices], dW_iter[train_idxs.indices] 
            S_iter_valid, dW_iter_valid = S_iter[val_idxs.indices], dW_iter[val_idxs.indices] 
            Y_iter_train = Y[train_idxs.indices]
            Y_iter_valid = Y[val_idxs.indices]

            start = time.time()
            iter_train_dataset = TensorDataset(S_iter_train, dW_iter_train, Y_iter_train)
            iter_valid_dataset = TensorDataset(S_iter_valid, dW_iter_valid, Y_iter_valid)
            train_loader = DataLoader(iter_train_dataset, batch_size=batch_size, shuffle=True)
            valid_loader = DataLoader(iter_valid_dataset, batch_size=batch_size_eval, shuffle=False)
            end = time.time()
            gamma1 = end - start

            start = time.time()
            for _ in range(num_epochs) :
                self.model.train()
                for S_batch, dW_batch , Y_batch in train_loader :
                    self.optimizer.zero_grad()
                    Fi = self.model(S_batch.unsqueeze(1))
                    loss = self.custom_loss(self.beta_dt*Y_batch,dW_batch, dW_batch*dW_batch, Fi)
                    loss.backward()
                    self.optimizer.step()

                self.model.eval()
                total_valid_loss = 0
                with torch.no_grad() :
                    for S_batch, dW_batch , Y_batch in valid_loader :
                        Fi = self.model(S_batch.unsqueeze(1))
                        loss = self.custom_loss(self.beta_dt*Y_batch,dW_batch, dW_batch*dW_batch, Fi)
                        total_valid_loss += loss.item()

                mean_valid_loss = total_valid_loss / len(valid_loader)
                early_stopping(mean_valid_loss)
                if early_stopping.early_stop:
                    print("Early stopping")
                    break
               
                

            end =  time.time()
            gamma2 = end - start            
            start = time.time()
            X,Y = self.eval_all(iter, S_iter, batch_size_eval, X ,  Y , dW_iter ) 
            end =  time.time()
            gamma3 = end - start
            # if write :
            #     print('data-loader part :', gamma1 )
            #     print(f' {num_epochs} epochs :',gamma2 )
            #     print(f' evall_all part  :', gamma3)
            #     write = False
         
        u0 = torch.mean(X)
        l0 = torch.mean(Y)
        return u0.item(),l0.item()

lr =  1e-3
Reccurent_Model = RecurrentNetworkModel(lr = lr , device = device)
num_epochs = 15
batch_size = 1024
batch_size_eval = 512

u0,l0 = Reccurent_Model.train(batch_size  = batch_size,batch_size_eval = batch_size_eval , num_epochs = num_epochs, valid_split = 0.2 , patience = 5 )

Processing Iterations:  32%|███▏      | 16/50 [02:54<06:09, 10.86s/it]

Early stopping


Processing Iterations:  38%|███▊      | 19/50 [03:21<05:09,  9.98s/it]

Early stopping


Processing Iterations:  40%|████      | 20/50 [03:26<04:17,  8.59s/it]

...

Processing Iterations:  96%|█████████▌| 48/50 [06:52<00:13,  6.75s/it]

Early stopping


Processing Iterations:  98%|█████████▊| 49/50 [07:01<00:07,  7.43s/it]

Early stopping


Processing Iterations: 100%|██████████| 50/50 [07:10<00:00,  8.60s/it]


In [49]:
print('The diff I found ', np.abs(u0-l0))
print('The diff they found in the article : ' ,4.4887-4.4762)

The diff I found  0.01656651496887207
The diff they found in the article :  0.01249999999999929
